## Задача

У нас есть некоторая выборка $X$ из распределения $p$ на бинарных векторах $v \in \{0, 1\}^m$. Наша задача в том, чтобы выучить это распределение. Выучить значит оценить $p$ и научиться из него семплировать. Также RBM позволит семплировать из всевозможных условных распределений типа $p(v_1, v_2 | v_3, ..., v_m)$, позволит, подобно автоэнкодеру или PCA, получать некоторое сжатое представление объектов. Наверное, есть ещё много применений.

## MNIST

<img src="13.png">

## MCMC. Gibbs sampling

Это набор техник, позволяющих семплировать из сложных распределений. Для обучения и применения RBM нам понадобится знать, что такое Gibbs sampling.

**Марковские цепи**

$p(x_1, ..., x_n) = p_1(x_1) \cdot p_2(x_2 | x_1) \cdot ... \cdot p_n(x_n | x_{n - 1})$ -- распределение каждого следующего состояния зависит только от предыдущего состояния и номера шага.

Теперь если $p_2 = p_3 = ... = p_n = q$, то марковская цепь называется гомогенной.
$p_2(x_2) = \int p(x_1, ..., x_n) dx_1 dx_3 ... dx_n = \int q(x_2 | x_1) p(x_1) dx_1 = Qp_1$ -- $Q$ линейный оператор, аналог умножения на матрицу для непрерывного случая. В дискретных марковских цепях, собственно, распределение на $n + 1$ шаге получается из распределения на $n$ шаге домножением на матрицу.

$p_n = Q^{n - 1} p_1$

Марковская цепь называется **эргодической**, если $\exists r: \forall p_1 \lim \limits_{n \rightarrow \inf} Q^n p_1 = r$.

**Теорема** Если $\forall x, y: q(y | x) > 0$, то марковская цепь -- эргодическая

**Утверждение** Если $\forall x, y$ выполнено соотношение $q(y | x) p(x) = q(x | y) p(y)$, то $p$ -- стационарное распределние для марковской цепи, задаваемой $q$. Соотношение называется уравнением детального баланса.

Действительно, 
<center>
    $(Qp)(y) = \int q(y | x) p(x) dx = \int q(x | y) p(y) dx = p(y)$
</center>

Теперь сформулируем более слабое достаточное условие эргодичности для случая конечного пространства состояний. Для непрерывного случая, формулировка аналогичная, но более громоздкая и нам не понадобится.

Марковская цепь является **неприводимой**, если для любой пары состояний существует ненулевая вероятность добраться из одного в другое за конечное число шагов, формально $\forall i, j \in \Omega\ \exists k > 0 : P(x_k = j | x_0 = i) > 0$ 

Марковская цепь является **апериодичной**, если для каждого состояния, моменты времени, в которые в нем можно оказаться распределены нерегулярно, формально $\forall i \in \Omega:\ GCD\{k | P(x_k = i | x_0 = i) > 0\} = 1$

**Теорема** Если марковская цепь неприводима и апериодична, а $r$ -- стационарное распределение, то  $\forall p_1 \lim \limits_{n \rightarrow \inf} Q^n p_1 = r$

Итак, суть методов MCMC в том, чтобы для $p$, из которого мы хотим семплировать, подобрать $q$, такое что выполняется уравнение детального баланса и выполняются достаточные условия эргодичности. Тогда начав из произвольной точки и переходя по Марковской цепи, начиная с некоторой итерации $i >> 1$ мы начнем получать семплы из распределения, очень близкого к $p$. 

**Gibbs sampling**

Пусть есть строго положительное распределение $\pi(x_1, ..., x_n)$ с конечным носителем $\Lambda^n$, и семплировать мы умеем только из $\pi(x_i | (x_v)_{v \in V \backslash i})$. Построим марковскую цепь, которая позволит нам семплировать из $\pi$. Для этого достаточно определить вероятность перехода.

Возмем любое строго положительное распределение вероятностей $q$ на $\{1, ..., n\}$. Пусть мы находимся в точке $(x_1^k, ..., x_n^k)$, разыграем индекс $i$ с помощью $q$, затем положим $\forall j \neq i:\ x_j^{k + 1} = x_j^k$, а $x_i^{k + 1}$ разыграем из $\pi(x_i | (x_v^k)_{v \in V \backslash i})$

<img src="1.png">

Проверка уравнения детального баланса:

<img src="2.png">

Неприводимость очевидна из строгой положительности $\pi, q$. Апериодичность из того, что $p_{xx} > 0$

Часто обходятся без $q$ и просто проходятся по индексам в порядке очереди, такая схема тоже сходится к заданному распределению (без доказательства).

## MRF

<img src="3.png">

В каждой вершине находится случайная величина, при этом выполнено локальное Марковское свойство $p(x_v | (x_w)_{w \in V \backslash \{v\}}) = p(x_v | (x_w)_{w \in N_v})$

<img src="4.png">

$E$ -- некоторая функция, которая зависит от параметра $\theta$, MRF можно обучать путем максимизации правдоподобия: $p(X | \theta)$. Правдоподобие же можно оптимизировать градиентным спуском. Для этого необходимо уметь считать градиент $\frac{\partial p}{\partial \theta}$ (для удоства работают с логправдоподобием):

<img src="5.png">

## RBM

RBM -- это MRF следующего вида:

<img src="6.png">

Так как энергия $E$ факторизуется по кликам (в данном графе клики -- всевозможные пары вершин $(h_i, v_j)$), то она имеет следующий вид:
<center>
    $E(h, v) = \sum \limits_{i, j} (d_{ij} h_i v_j + e_{ij} (1 - h_i) v_j + s_{ij} h_i (1 - v_j) + t_{ij} (1 - h_i) (1 - v_j))$, и с точностью до константы, которая нас не волнует<br>
    $E(h, v) = \sum \limits_{i, j} w_{ij} h_i v_j + \sum \limits_j b_j v_j + \sum \limits_i c_i h_i$
</center>

Выражения для рассчета условных вероятностей на видимые/скрытые переменные:

<img src="7.png">
<img src="8.png">

Посчитаем градиент логправдоподобия:

<img src="9.png">
<img src="10.png">
<img src="11.png">

Теперь единственная проблема -- научиться считать матожидание по $p(v)$, утверждается, что это можно эффективно сделать по следующей схеме:

<img src="12.png">

где $v^{(k)}$ -- семпл, полученный на $k$-ой итерации Gibbs sampling из $v^{(0)}$

In [1]:
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn import datasets
from tqdm import tqdm
from copy import deepcopy
from functools import partial

In [2]:
def gibbs_sampling_generator(sampler, state, steps):
    for i in range(steps):
        state = sampler.sample(state, i % len(state))
        yield state
    

def range_gibbs_sampling(sampler, state, begin, end):
    return list(gibbs_sampling_generator(sampler, state, end))[begin:]


def gibbs_sampling(sampler, state, steps):
    return range_gibbs_sampling(sampler, state, steps - 1, steps)[0]

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def h_ps(model, v):
    return sigmoid(model.W @ v + model.c)


class BlockSampler:
    def __init__(self, model, active_index=None):
        self.model = model
        self.active_index = active_index
    
    def sample(self, state, index):
        state = deepcopy(state)
        
        h, v = state
        
        if index == 0:
            ps = h_ps(self.model, v)
        else:
            ps = sigmoid(self.model.W.T @ h + self.model.b)
        
        ai = self.active_index[index] if self.active_index else slice(None)
        state[index][ai] = sps.bernoulli.rvs(ps[ai])
        
        return state

In [4]:
class RBM:
    def __init__(self, W, b, c):
        self.W = W
        self.b = b
        self.c = c
    
    def __add__(self, other):
        return RBM(self.W + other.W, self.b + other.b, self.c + other.c)
    
    def __mul__(self, other):
        return RBM(self.W * other, self.b * other, self.c * other)
    
    __rmul__ = __mul__
    
    def __sub__(self, other):
        return self + other * -1
    
    def __truediv__(self, other):
        return self * (1 / other)

In [5]:
def numerator_grad(sample, model):
    ps = h_ps(model, sample)
    
    return RBM(ps @ sample.T, sample, ps)


def compute_grad(num_gibbs_steps, sample, model):
    bs = BlockSampler(model)
    _, gibbs_sample = gibbs_sampling(bs, [np.zeros_like(model.c), sample], num_gibbs_steps)

    return numerator_grad(sample, model) - numerator_grad(gibbs_sample, model)


def sgd(model, compute_grad, samples, batch_size, lr, steps):
    for _ in tqdm(range(steps)):
        batch_index = np.random.choice(np.arange(samples.shape[0]), size=batch_size, replace=False)
        batch = samples[batch_index]
        
        grad = np.mean([compute_grad(sample.T, model) for sample in batch])
        model = model + lr * grad
        
    return model

In [6]:
def predict(model, samples, num_classes, sampling_steps):
    bs = BlockSampler(model, [slice(None), slice(samples.shape[1], None)])
    samples = np.hstack([samples, np.zeros((samples.shape[0], num_classes))])
    answer = []
    
    for sample in tqdm(samples):
        states = range_gibbs_sampling(bs, [np.zeros_like(model.c), sample.T], 
                                      sampling_steps * 2 // 3, sampling_steps)
        visibles = np.squeeze(np.array([v for h, v in states]))
        answer.append(np.mean(visibles[:, -num_classes:], axis=0).argmax())
    
    return np.array(answer)

In [7]:
mnist = datasets.load_digits()

In [8]:
def otsu_impl(image, num_bins=16):
    pixels = image.ravel()
    
    borders = np.linspace(np.min(pixels), np.max(pixels), num_bins, endpoint=False)
    bins = np.digitize(pixels, borders)
    
    values, counts = np.unique(bins, return_counts=True)

    all_counts = np.zeros((num_bins,))
    all_counts[values - 1] = counts
    probs = all_counts / all_counts.sum()
    
    omega = np.cumsum(probs)
    mu = np.cumsum(np.arange(1, num_bins + 1) * probs)

    omega_cut = omega[:-1]
    mu_cut = mu[:-1]
    sigma = ((mu[-1] * omega_cut - mu_cut) ** 2) / (omega_cut * (1 - omega_cut))
    k_opt = np.argmax(sigma)
    
    binarized_mask = np.ones_like(pixels)
    binarized_mask[pixels < borders[k_opt + 1]] = 0
    binarized_mask = binarized_mask.reshape(image.shape[:2])
    
    return binarized_mask

In [9]:
num_samples = mnist['images'].shape[0]
images = np.asmatrix(np.array([otsu_impl(image) for image in mnist['images']]).reshape((num_samples, -1)))
target = np.zeros((num_samples, 10))
target[np.arange(num_samples), mnist['target']] = 1

train_size = 1500

In [10]:
data = np.asmatrix(np.hstack([images, target]))

In [11]:
def initial_model(visible, hidden):
    W = np.asmatrix(np.random.normal(size=(hidden * visible)).reshape(hidden, visible))
    b = np.asmatrix(np.zeros((visible,))).T
    c = np.asmatrix(np.zeros((hidden,))).T
    
    return RBM(W, b, c)

In [12]:
rbm = initial_model(data.shape[1], 500)

In [13]:
model = sgd(rbm, partial(compute_grad, 10), data[:train_size], 100, 1, 1000)

100%|██████████| 1000/1000 [08:14<00:00,  2.02it/s]


In [14]:
answer = predict(model, images[train_size:], 10, 300)

100%|██████████| 297/297 [00:37<00:00,  7.88it/s]


In [15]:
np.mean(answer == target[train_size:].argmax(axis=1))

0.898989898989899

## То же самое нейронкой

In [16]:
images_std = images.std(axis=0)
images_std[images_std == 0] = 1

normalized_images = (images - images.mean(axis=0)) / images_std

In [17]:
batch_size = 50

In [18]:
tf.reset_default_graph()

input_placeholder = tf.placeholder(dtype=tf.float32, shape=(batch_size, images.shape[1]))
target_placeholder = tf.placeholder(dtype=tf.int32, shape=(batch_size, 10))

output = tf.nn.relu(tf.layers.dense(input_placeholder, 50))
output = tf.nn.relu(tf.layers.dense(output, 40))
output = tf.nn.relu(tf.layers.dense(output, 20))
output = tf.layers.dense(output, 10)
result = tf.argmax(output, axis=1)

loss = tf.losses.softmax_cross_entropy(target_placeholder, output)
optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

In [19]:
steps = 5000

In [20]:
train_X = normalized_images[:train_size]
train_target = target[:train_size]

In [21]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

batch_losses = []

for _ in range(steps):
    batch_index = np.random.choice(np.arange(train_X.shape[0]), size=batch_size, replace=False)
    batch = train_X[batch_index]
    batch_target = train_target[batch_index]

    batch_loss, _ = sess.run([loss, optimizer], feed_dict={
        input_placeholder: batch, target_placeholder: batch_target})
    
    batch_losses.append(batch_loss)
    
    if len(batch_losses) == 1000:
        print(np.mean(batch_losses))
        batch_losses = []

0.07115312
6.408492e-05
1.8792354e-05
9.001453e-06
4.4420854e-06


In [22]:
answer = []

for image in normalized_images[train_size:]:
    batch = np.vstack([image, normalized_images[:batch_size - 1]])
    answer.append(sess.run(result, feed_dict={input_placeholder: batch})[0])

In [23]:
np.mean(answer == target[train_size:].argmax(axis=1))

0.8552188552188552